### Gather data for PCA

In [5]:
import pandas as pd
import geopandas as gpd
import numpy as np
from rasterio import rasterio, Affine
from rasterio.warp import reproject, Resampling
from pathlib import Path
from scipy import ndimage
from scipy.misc import bytescale


In [6]:
#inputs local
inortho = r"T:\UAS\2018-676-FA\products\ortho\2018-10-23-Wildlands_ortho.tif"
indsm = r"T:\UAS\2018-676-FA\products\dsm\2018-10-23-Wildlands_DSM_10cm.tif"
invari = r"D:\jlogan\2018-676-FA\tmp_veg_indices\vari_scaled_int_fromlzw.tif"
intgi = r"D:\jlogan\2018-676-FA\tmp_veg_indices\tgi_scaled_int_fromlzw.tif"
inpoints = r"T:\UAS\2018-676-FA\validation\wld_topo_17-18_combined_kdthinned10cm.csv"

#inputs lab computer
#inortho = r"D:\jlogan\2018-676-FA\products\ortho\2018-10-23-Wildlands_ortho_lzw.tif"
#indsm = r"D:\jlogan\2018-676-FA\products\dsm\2018-10-23-Wildlands_DSM_10cm.tif"
#invari = r"D:\jlogan\2018-676-FA\tmp_veg_indices\vari_scaled_int_fromlzw.tif"
#intgi = r"D:\jlogan\2018-676-FA\tmp_veg_indices\tgi_scaled_int_fromlzw.tif"
#inpoints = r"T:\UAS\2018-676-FA\validation\wld_topo_17-18_combined_kdthinned10cm.csv"

for n in [inortho, indsm, invari, intgi, inpoints]:
    n = Path(n)   

In [7]:
#load input pts
ptdf = pd.read_csv(inpoints)

In [8]:
#load input rgb image
dataset = rasterio.open(inortho)
imgaff = dataset.transform
#img = dataset.read()

In [5]:
r = dataset.read(1)
g = dataset.read(2)
b = dataset.read(3)

In [6]:
#calc psuedo veg indices
#https://agribotix.com/blog/2017/04/30/comparing-rgb-based-vegetation-indices-with-ndvi-for-agricultural-drone-imagery/
tgi = g - 0.39 * r - 0.61 * b
vari = (g-r)/g+r-b

#mask infs in vari
vari_masked = np.ma.masked_invalid(vari)
tgi_masked = np.ma.masked_invalid(tgi)
del tgi, vari

#scale to 0 -255
vari_scaled = bytescale(vari_masked)
tgi_scaled = bytescale(tgi_masked)
del tgi_masked, vari_masked


C:\Users\jlogan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
C:\Users\jlogan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
C:\Users\jlogan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `bytescale` is deprecated!
`bytescale` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
  if sys.path[0] == '':
C:\Users\jlogan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: `bytescale` is deprecated!
`bytescale` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
  del sys.path[0]


In [7]:
r.shape, vari_scaled.shape

((43008, 30720), (43008, 30720))

In [9]:
#for r, g, b, vari and tgi sample grid and add to df

#get img coords on df
ptdf['imgcol_int'], ptdf['imgrow_int'] = np.floor(~imgaff * (ptdf['e'], ptdf['n'])).astype(int)

ptdf['r'] = r[ptdf['imgrow_int'],ptdf['imgcol_int']]
ptdf['g'] = g[ptdf['imgrow_int'],ptdf['imgcol_int']]
ptdf['b'] = b[ptdf['imgrow_int'],ptdf['imgcol_int']]
ptdf['vari'] = vari_scaled[ptdf['imgrow_int'],ptdf['imgcol_int']]
ptdf['tgi'] = tgi_scaled[ptdf['imgrow_int'],ptdf['imgcol_int']]

In [9]:
#load dsm
dataset = rasterio.open(indsm)
dsmaff = dataset.transform
dsm = dataset.read()
dsm.shape

(1, 13470, 9630)

In [ ]:
#resample to match input rgb image
#from https://github.com/mapbox/rasterio/blob/master/docs/topics/resampling.rst
orig_dsm_res = dsmaff.a
target_dsm_res = imgaff.a
resample_factor = orig_dsm_res / target_dsm_res

dsm_resamp = np.empty(shape=(dsm.shape[0],  # same number of bands
                         round(dsm.shape[1] * resample_factor), # new resolution
                         round(dsm.shape[2] * resample_factor)))

# adjust the new affine transform to the smaller cell size
newaff = Affine(dsmaff.a / resample_factor, dsmaff.b, dsmaff.c,
                dsmaff.d, dsmaff.e / resample_factor, dsmaff.f)

print(f'Orig. cell size: {dsmaff.a}\nTarget cell size: {imgaff.a}\nOutput cell size: {newaff.a}')

reproject(
    dsm, dsm_resamp,
    src_transform = dsmaff,
    dst_transform = newaff,
    src_crs = dataset.crs,
    dst_crs = dataset.crs,
    resampling = Resampling.bilinear)

In [ ]:
#get new pt coords for newdsm
ptdf['dsmcol'], ptdf['dsmrow'] = ~newaff * (ptdf['e'], ptdf['n'])

In [ ]:
#generic filter not working.  very slow
# #calc mean of kernel
# #from https://gis.stackexchange.com/a/254795/129277
# # use 10 cm DSM for less memory requirements 
# #(just use 1/3 * desired kernel size (eg. 96 pixel kernel on 3cm dsm would be 32 pixel kernel))
# std96 = ndimage.generic_filter(dsm, np.nanstd, size=32, mode='constant', cval=np.NaN)


In [ ]:
#get elevation mean and std over 96 and 128 pixel windows
def getstdev(dsmcol, dsmrow, dsm, numpixels):
    intcol = np.floor(dsmcol).astype(int)
    introw = np.floor(dsmrow).astype(int)
    halfkernel = numpixels / 2
    kern_std = np.nanstd(dsm[introw-halfkernel:introw+halfkernel, intcol-halfkernel:intcol+halfkernel])
    return(kern_std)

ptdf['std96'] = ptdf.apply(lambda row: getstdev(row['dsmcol'], row['dsmrow'], dsm_resamp, 96), axis=1)
ptdf['std128'] = ptdf.apply(lambda row: getstdev(row['dsmcol'], row['dsmrow'], dsm_resamp, 128), axis=1)

def getmean(dsmcol, dsmrow, dsm, numpixels):
    intcol = np.floor(dsmcol).astype(int)
    introw = np.floor(dsmrow).astype(int)
    halfkernel = numpixels / 2
    kern_mean = np.nanmean(dsm[introw-halfkernel:introw+halfkernel, intcol-halfkernel:intcol+halfkernel])
    return(kern_mean)

ptdf['mean96'] = ptdf.apply(lambda row: getmean(row['dsmcol'], row['dsmrow'], dsm_resamp, 96), axis=1)
ptdf['mean128'] = ptdf.apply(lambda row: getmean(row['dsmcol'], row['dsmrow'], dsm_resamp, 128), axis=1)

In [69]:
xy = np.floor(~dsmaff * (ptdf.loc[0]['e'], ptdf.loc[0]['n'])).tolist()
print(dsm[0,int(xy[1]),int(xy[0])])
xy2 = np.floor(~newaff * (ptdf.loc[0]['e'], ptdf.loc[0]['n'])).tolist()
print(dsm_resamp[0,int(xy2[1]),int(xy2[0])])

2.9251773
2.925618998994598
